# 浮動小数点数型: `Float64`

## 指数表記
- 計算機において小数は **浮動小数点数 (Floating Point Number)** として実装されている．
- 例えば，10進数の浮動小数点数は $1.234 \times 10^{56}$ のように指数表記（仮数部✕指数部）で表されるような数値を表す．
- ただし，仮数部の桁数は固定で，指数の範囲も有界である．
   
<img src="fpn.svg">

In [1]:
1024.0^3 # 1Gibi 

1.073741824e9

- `e9` は $10^9$ に相当する． 大文字`E`でも構わない．

In [2]:
1.234E9

1.234e9

- `e`の直後に負数を書くと減算ではなく，負べきになる．

In [3]:
1e-8   # 10^{-8}

1.0e-8

## 2進浮動小数点数
- 計算機では数値は **2進数** で実装されている．これ以降は浮動小数点数の**base は2**とする．
- 2進数表記された数値には $1011_2 = 7$　のように添字に2をつけることにする．
- 10進数で分数や小数で数値を記述したときに，2進数に変換されるので，微小ながらも誤差が発生する．
- 例えば，$1/8$ などは $0.001_2$ なので正確に変換される．
- 0.1 は2進数では無限小数：
$$
0.1 = 2^{-4} + 2^{-5} + 2^{-8} + 2^{-9} + \cdots + 2^{-4n} + 2^{-(4n+1)} + \cdots 
= 0.000110011001100_2 \cdots
$$
となるので，有限桁では正確に表現できない．

In [4]:
x  = 0.1

0.1

-  これは一見すると正確に変換されているように思えてるが，0.2を足してみると誤差が生じていることが確認できる．

In [5]:
x += 0.2

0.30000000000000004

## 浮動小数点数の標準規格

- 浮動小数点数の標準規格はIEEE754で策定されており，次のような数や特殊値で構成される．
    - 正規化数 (normal number)
    - 非正規化数 (subnormal number)
    - 特殊な値：`Inf` （無限大） ， `NaN` (Not a Number) など．    

## 単精度と倍精度

- 多くの計算機でハードウェアレベルで実装されている浮動小数点数は次の２種類：
     - 単精度 (single precision) ：2進数23桁・・・`Float32`
     - 倍精度 (double precision) ：2進数52桁・・・`Float64`
- デフォルトでは倍精度が用いられる．→ 授業でも倍精度を使ってください．
- 普及帯のグラフィックボードに搭載されているGPUは単精度計算のみをサポートしている．
- 科学技術計算用途のGPUでは倍精度がサポートされているものもある．

## 正規化数，非正規化数
- ここからは，**倍精度**浮動小数点数のみを考える．

- 正規化数とは，$1.d_1 d_2 \cdots d_{52} \times 2^e$ という先頭桁が1の小数である．仮数部は2進数表示である．
    - $d_1, d_2, \ldots, d_{52} \in \{0, 1\}$
    - $-1022 \le e \le 1023$
    
- 非正規化数は，$0.d_1 d_2 \cdots d_{52} \times 2^{-1023}$の形で表される小数のことである．
- 正規化数を数直線上にプロットすると原点付近で相対的に大きな隙間が生まれる．
- 非正規化数はこの隙間を均質に埋める役割を担っている．

<img src = "fpn01.png" width="80%">

## 丸め誤差
- 浮動小数点数どうしを四則演算して得られた値は浮動小数点数になるとは限らない．

In [6]:
1.111111111^2   

1.2345679009876545

- 正確な値は $1.111111111^2 = 1.234567900987654321$ であり，この場合は下4桁分が切り捨てられている．
- このように演算結果に最も近い浮動小数点数への近似を **丸め** (rounding) といい，その際に生じる誤差を **丸め誤差** (rounding error)  という．
- 丸め方には，正/負の無限大の方向，原点方向，最近点への丸めなどがある．
- 丸め誤差は浮動小数点数の計算において，ほぼ必然的に発生する．微小ではあるが，大きな誤差を発生させることもある．


## Overflow
- 倍精度で扱える最大の正の正規化数は`floatmax(Float64)`で取得できる．

In [7]:
x_max = floatmax(Float64)

1.7976931348623157e308

- 浮動小数点数の演算の結果が `x_max`を超えた場合，無限大を表す `Inf` に変換される．
このような現象を overflow という．

In [8]:
2 * x_max 

Inf

#### 📝最大の正規化数
- `x_max`は $1.11\cdots 1_2 \times 2^{1023}$に等しい．
- `BigFloat`で計算して一致することを確認してみよう．
- 仮数部を10進数に直すと，
$$
2^{0} + 2^{-1} + \cdots + 2^{-53} = \frac{1 - 2^{-54}}{1- 2^{-1}}
 = 2 - 2^{-54}.
$$

In [9]:
(2 - big(2.0)^(-54))*big(2.0)^1023

1.797693134862315857833297452421029442208659153021342421048019300690888220491329e+308

- 仮数部は15桁まで，指数部は一致している．

#### 📝最小の正の正規化数
- 倍精度で扱える **最小の正の正規化数** を知りたい場合は，`floatmin`を使う．

In [10]:
x_min = floatmin(Float64)

2.2250738585072014e-308

## Underflow
- 最小の正の**非**正規化数（正規化数ではない）は `eps(0.0)` で取得できる．

In [11]:
subnormal_min = eps(0.0)

5.0e-324

---
- 【注意】正しい値は $0.00\cdots 01_2 \times 2^{-1022} = 4.940\cdots \times 10^{-324}$なので，表示されている数値は誤りです．
内部的には正しい値が格納されているようなので，表示上のバグかもしれません．原因が分かる人は教えてください．

In [12]:
bitstring(subnormal_min) # bit列の表示．これは問題ない．

"0000000000000000000000000000000000000000000000000000000000000001"

---

- 演算結果が この値を下回ると `0.0` に丸められることがあり，これを underflow という．

In [13]:
subnormal_min/2  

0.0

#### 📝 `subnormal_min`の確認
- `subnormal_min` は $0.00\cdots 01_2 \times 2^{-1022}$に等しい．
- 仮数部は10進数で表すと，$2^{-52}$だから，$2^{-(1022 + 52)} = 2^{-1074}$.
- `BigFloat`で表示してみる．

In [14]:
big(2.0)^(-1074)

4.940656458412465441765687928682213723650598026143247644255856825006755072702088e-324

In [15]:
big(eps(0.0))   # BigFloatで確認．一致する．

4.940656458412465441765687928682213723650598026143247644255856825006755072702088e-324

## `NaN` (Not a Number)

- 浮動小数点数において未定義な演算等を行った結果，`NaN` という特殊な値が発生することがある．
- 例えば，0を0で割ると `Nan` が発生する．

In [16]:
0/0

NaN

- ちなみに0以外の正の浮動小数点数をゼロで割ると`Inf`になる．

In [17]:
1/0

Inf

- `Inf`に関する四則演算の結果，`NaN` が発生することがある． 

In [18]:
@show Inf + Inf
@show Inf - Inf    # ∞ - ∞
@show Inf * Inf    
@show Inf * 0      # ∞ ✕ 0
@show Inf / Inf    # ∞ ÷ ∞

Inf + Inf = Inf
Inf - Inf = NaN
Inf * Inf = Inf
Inf * 0 = NaN
Inf / Inf = NaN


NaN

- 不定形の極限に対応するような演算を行うと `NaN` が発生すると覚えておけばよい．

#### Warning
- `Inf` や `NaN` の発生を前提としたコード設計はしないようにしてください．
`Inf` や `NaN`の挙動を完全に把握することは非常に困難で，bug （プログラムにおける予期しない動作やエラー）
を生みやすいです．さらに，読み手にとっても理解しにくいコードになります．
- Juliaでは `isfinite(x)` で `x` が有限値であるかどうか， `isnan(x)`で `x` が `NaN` であるかどうかの判定ができます．
    

## 情報落ち
- 浮動小数点数では，絶対値が machine epsilonより小さな値を加算しても，丸めによって値が変化しないことがある．
これを**情報落ち**という．

- 倍精度のmachine epsilonは `eps(Float64)` or `eps(1.0)`などで取得できる．

In [19]:
ϵ = eps(Float64)

2.220446049250313e-16

In [20]:
1.0 + 1e-16

1.0

- 上のような例では情報落ちを回避することはできないが，
計算順序を変えることで情報落ちを回避できることがある．

In [21]:
(1e-16 + 1.0) - 1.0   # 情報落ちが発生する

0.0

In [22]:
1e-16 + (1.0 - 1.0)   # 情報落ちは発生しない

1.0e-16

##  桁落ち
- 近い値の２つの浮動小数点数に関して引き算をすると有効桁数が減少する（桁落ち）．
- 例えば，$1.2345 - 1.2344 = 0.0001 = 1 \times 10^4$ は，有効桁数5桁どうしを引き算した結果，有効桁数が1桁に減っている．
- したがって，浮動小数点数の計算では近い値を持つもの同士の減算はできる限り避けたほうがよい．  

## 桁落ちの例と回避法

- $b$ を正の実数とする．
- 2次方程式　$f(x) = x^2 - 2bx + 1 = 0$ の解は，解の公式より
$$
  \alpha := b - \sqrt{b^2 - 1}, \quad \beta := b + \sqrt{b^2 - 1} 
$$
と表される．

- $b^2 \gg 1$の場合，$\alpha$をこのまま計算すると， $\sqrt{b^2 - 1} \approx b$ であるから桁落ちが生じる．

In [23]:
b = 1e8  # 1e8 = 10^8 
alpha = b - sqrt(b^2-1)

0.0

- 計算値は `0.0` であるが，これが解ではないことは代入してみれば明らかである．
- このような場合でも，式変形で桁落ちを避けることができる．
$$
b - \sqrt{b^2 - 1}  = \frac{1}{b + \sqrt{b^2 - 1}} 
$$

- 上の式で計算してみると，真値に近い値が得られる．

In [24]:
b = 1e8
x = 1/(b + sqrt(b^2-1))

5.0e-9

#### Remark
- 上の $\alpha$ の計算式は万能ではなく，$b<0$ かつ $b^2 \gg 1$ の場合は桁落ちが発生する．
    

#### 📝 `bitstring()`によるFloat64のbit列表示
- 浮動小数点数は計算機内部では2進数で実装されているが，プログラムコードではわかりやすさのために10進数で表記される．
- 倍精度浮動小数点数がどのように2進数（bit列）で格納されているか見てみよう．
- 最も簡単な例として，`1.0`の場合：

In [25]:
bitstring(1.0)

"0011111111110000000000000000000000000000000000000000000000000000"

- 先頭のbitは符号が格納されている．`0`はプラス，`1`はマイナスである．`-1.0`のbit列は先頭が`1`になる．

In [26]:
bitstring(-1.0)

"1011111111110000000000000000000000000000000000000000000000000000"

- 2〜12bitに格納されている値 $e$ は指数に対応している．
- $e$ は $0$ 以上 $2047 = 1 11111 11111_2 $以下の整数値を取りうる．
- 次のような規則で指数あるいは特殊値に変換される．
    - $e=0$:  ゼロ（仮数部が全て0) or 非正規化数（仮数部が非ゼロ)
    - $1 \le e \le 2046$: $e - 1023$ が指数．
    - $e = 2047$: `Inf`,`-Inf` (仮数部が全て0) or `NaN`（仮数部が非ゼロ)

- 【例】
    - `bitstring(1.0)` の場合は $e=0111111111_2 = 1023$ だから指数は $e-1023 = 0$である．
    - 仮数部の情報は13bit以降に格納されている．

- $1+2^{-5} = 1.0001$ (2進数) の場合：

In [27]:
bitstring(1+2^-5)[13:end]

"0000100000000000000000000000000000000000000000000000"

- 仮数部のbit列は（先頭の桁は省略されて）`0000100...0`となっている．
- $0.1$ (10進数） は有限小数であるが，2進数では無限小数となる．
$0.1$ を倍精度に変換すると，途中で桁が打ち切られるので，丸め誤差が生じる．

In [28]:
bitstring(0.1)[13:end]

"1001100110011001100110011001100110011001100110011010"

- 以下，特徴的な浮動小数点数のbit列を表示してみる．

In [29]:
bitstring(0.0)  # ゼロ

"0000000000000000000000000000000000000000000000000000000000000000"

In [30]:
bitstring(-0.0)  # 負のゼロ

"1000000000000000000000000000000000000000000000000000000000000000"

In [31]:
bitstring(floatmax(Float64))  # 最大の正の正規化数＝1.111...11e1023

"0111111111101111111111111111111111111111111111111111111111111111"

In [32]:
bitstring(eps(0.0)) # 最小の正の非正規化数＝0.000...01e-1022

"0000000000000000000000000000000000000000000000000000000000000001"

In [33]:
bitstring(Inf)  #形式的には 1.000...00e1024

"0111111111110000000000000000000000000000000000000000000000000000"

In [34]:
bitstring(-Inf)  # Infとは符号bitのみ異なる

"1111111111110000000000000000000000000000000000000000000000000000"

In [35]:
bitstring(NaN)   # 形式的には 1.100.00e1024

"0111111111111000000000000000000000000000000000000000000000000000"